In [35]:
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, experimental, MaxPool2D
from keras.callbacks import History, EarlyStopping 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.random import set_seed
from tensorflow.keras.utils import to_categorical
import plotly.express as px


In [36]:
# images will be classified as cat (0) or dog (1)
categories = ['cats', 'dogs']
directory = '../input/dogs-cats-images/dataset/training_set'

In [37]:
IMG_SIZE = (150, 150)
# list of image and labels
data = []

# get path to each image
for category in categories:
    folder = os.path.join(directory, category)
    label = categories.index(category)
    for img in os.listdir(folder):
        # ignore incompatible images (gifs in this dataset)
        try:
            img_path = os.path.join(folder, img)
            # convert image into array
            img_arr = cv2.imread(img_path)
            # reshape image
            img_arr = cv2.resize(img_arr, IMG_SIZE)
            # putting image and labels into list
            data.append([img_arr, label])
        except Exception as e:
            print(str(e))
            print(img_path)

In [38]:
len(data)

8000

In [39]:
# shuffle the data
random.shuffle(data)
# show first image array
data[0]

[array([[[ 27,  51,  58],
         [ 20,  49,  60],
         [ 14,  47,  63],
         ...,
         [ 80, 100, 106],
         [ 84, 106, 106],
         [162, 167, 162]],
 
        [[ 19,  45,  51],
         [ 18,  48,  60],
         [ 17,  50,  66],
         ...,
         [ 79,  99, 105],
         [ 79, 101, 101],
         [154, 158, 155]],
 
        [[ 26,  52,  58],
         [ 17,  48,  59],
         [ 17,  51,  67],
         ...,
         [ 84, 104, 111],
         [ 84, 106, 108],
         [149, 153, 151]],
 
        ...,
 
        [[ 37,  51,  53],
         [ 30,  45,  48],
         [ 37,  52,  58],
         ...,
         [ 71,  80,  91],
         [ 72,  82,  87],
         [147, 158, 156]],
 
        [[ 79,  89,  87],
         [ 49,  59,  59],
         [ 62,  73,  75],
         ...,
         [ 91,  97, 104],
         [ 90,  96,  97],
         [150, 156, 151]],
 
        [[193, 199, 194],
         [168, 175, 171],
         [174, 183, 181],
         ...,
         [177, 181, 184],
  

In [40]:
# lists of images and labels
X = []
y = []

for features, labels in data:
    X.append(features)
    y.append(labels)

In [41]:
# convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

In [42]:
# normalizing data
X = X/255

In [43]:
X.shape

(8000, 150, 150, 3)

In [44]:
# Define the model architecture
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), input_shape=X.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Conv2D(512, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(loss="sparse_categorical_crossentropy",
          optimizer='adam',
          metrics=['accuracy'])

In [45]:
early_stop = EarlyStopping(monitor="val_loss",patience=5,mode="min") # Ensure the model doesn't overfit

history = model.fit(X, y, epochs=15, validation_split=0.1)

Epoch 1/15
225/225 [==============================] - 13s 56ms/step - loss: 1.7146 - accuracy: 0.5701 - val_loss: 0.6895 - val_accuracy: 0.5537
Epoch 2/15
225/225 [==============================] - 12s 53ms/step - loss: 0.6038 - accuracy: 0.6784 - val_loss: 0.6850 - val_accuracy: 0.5938
Epoch 3/15
225/225 [==============================] - 12s 53ms/step - loss: 0.5129 - accuracy: 0.7488 - val_loss: 0.5723 - val_accuracy: 0.6938
Epoch 4/15
225/225 [==============================] - 12s 53ms/step - loss: 0.4478 - accuracy: 0.7855 - val_loss: 0.6628 - val_accuracy: 0.6800
Epoch 5/15
225/225 [==============================] - 12s 53ms/step - loss: 0.4004 - accuracy: 0.8229 - val_loss: 0.5675 - val_accuracy: 0.7200
Epoch 6/15
225/225 [==============================] - 12s 53ms/step - loss: 0.3308 - accuracy: 0.8566 - val_loss: 0.5802 - val_accuracy: 0.7212
Epoch 7/15
225/225 [==============================] - 12s 53ms/step - loss: 0.3055 - accuracy: 0.8692 - val_loss: 0.5193 - val_accuracy:

In [46]:
# Generate generalization metrics
fig = px.line(
history.history, y=['loss', 'val_loss'],
labels={'index': 'epoch', 'value': 'loss'}, 
title='Training History Loss')
fig.show()

In [ ]:
fig = px.line(
history.history, y=['accuracy', 'val_accuracy'],
labels={'index': 'epoch', 'value': 'accuracy'}, 
title='Training History Accuracy')
fig.show()